In [ ]:
!pip install efficientnet -q

In [ ]:
!pip install wandb --upgrade

In [ ]:
import tensorflow_addons as tfa

In [ ]:
import os

import efficientnet.tfkeras as efn
import numpy as np
import pandas as pd
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.model_selection import GroupKFold

import csv
from kaggle_datasets import KaggleDatasets

from tensorflow.keras import layers

In [ ]:
import wandb
from wandb.keras import WandbCallback
api_key = 'e5260514cd613164ad5c3fd66ad7e909beaf820f'
wandb.login(key=api_key)

In [ ]:
"""
import cv2
import albumentations
from albumentations import (
    Compose, HorizontalFlip, CLAHE, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma,
    ToFloat, ShiftScaleRotate
)

AUGMENTATIONS_TRAIN = Compose([
    HorizontalFlip(p=0.5),
    RandomContrast(limit=0.2, p=0.5),
    RandomGamma(gamma_limit=(80, 120), p=0.5),
    RandomBrightness(limit=0.2, p=0.5),
    HueSaturationValue(hue_shift_limit=5, sat_shift_limit=20,
                       val_shift_limit=10, p=.9),
    # CLAHE(p=1.0, clip_limit=2.0),
    ShiftScaleRotate(
        shift_limit=0.0625, scale_limit=0.1, 
        rotate_limit=15, border_mode=cv2.BORDER_REFLECT_101, p=0.8), 
    #ToFloat(max_value=255)
])
"""

In [ ]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy


def build_decoder(with_labels=True, target_size=(256, 256), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)

        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")
        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode

In [ ]:
#with strategy.scope():
#    data_augmentation = tf.keras.Sequential([
#        layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
#        layers.experimental.preprocessing.RandomRotation(0.2),
#        layers.experimental.preprocessing.RandomContrast(0.025)
#    ])

In [ ]:
import random

In [ ]:
def build_augmenter(with_labels=True):
    def augment(img):
        #print("img.shape", img.shape)
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        
        #img = tf.image.random_saturation(img, 0.5, 1.0)
        #img = tf.image.random_saturation(img, 0.2, 0.5)
        #img = tf.image.random_brightness(img, 0.2)
            
        #img = AUGMENTATIONS_TRAIN(image=img)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=128, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) 
    
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize)
    
    #if augment:
    #    dset = dset.map(lambda x, y: (data_augmentation(x, training=augment), y), 
    #                num_parallel_calls=AUTO)
    
    dset = dset.prefetch(AUTO)
    
    return dset

In [ ]:
#train_dataset

In [ ]:
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 8

In [ ]:
COMPETITION_NAME = "covid19512"
GCS_DS_PATH = KaggleDatasets().get_gcs_path(COMPETITION_NAME)

In [ ]:
_data_dir = GCS_DS_PATH
load_dir = _data_dir + "/train/"
df = pd.read_csv(_data_dir + '/updated_iamge_level.csv')

In [ ]:
df.head()

In [ ]:
df['none'] = 0 
for i, r in df.iterrows():
    if r.label == "none 1 0 0 1 1":
        df.loc[i, 'none'] = 1
#df

In [ ]:
_duplicateList_path = '../input/covid19512/dublicate.txt'
_duplicateList = []
with open(_duplicateList_path, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        _duplicateList += row

_duplicateList[:5]

In [ ]:
df['id'] = df['id'].str.replace("_image", "")
df = df[~df['id'].replace().isin(_duplicateList)]

In [ ]:
IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600, 512)
IMS = 8

In [ ]:
e = efn.EfficientNetB3(
                input_shape=(IMSIZE[IMS], IMSIZE[IMS], 3),
                weights='imagenet',
                include_top=False)
e.summary()

In [ ]:
for i in range(5):
    
    config={
        "fold": i,
        "start_lr": 1e-3,
        "min_lr":1e-6,
        "epoch": 20,
        "batch_size": BATCH_SIZE,
        "IMG_SIZE": IMSIZE[IMS]
    }
    wandb.init(project="covid19-siim-2class", config=config)
    
    valid_paths = load_dir + df[df['fold'] == i]['id'] + '.png' #"/train/"
    train_paths = load_dir + df[df['fold'] != i]['id'] + '.png' #"/train/" 
    valid_labels = df[df['fold'] == i]['none'].values
    train_labels = df[df['fold'] != i]['none'].values

    

    decoder = build_decoder(with_labels=True, target_size=(IMSIZE[IMS], IMSIZE[IMS]), ext='png')
    test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[IMS], IMSIZE[IMS]),ext='png')

    train_dataset = build_dataset(
        train_paths, train_labels, bsize=BATCH_SIZE, decode_fn=decoder
        # 使用 sequence augment
        #, augment=False
    )

    valid_dataset = build_dataset(
        valid_paths, valid_labels, bsize=BATCH_SIZE, decode_fn=decoder,
        repeat=False, shuffle=False, augment=False
    )

    try:
        n_labels = train_labels.shape[1]
    except:
        n_labels = 1

    with strategy.scope():
        model = tf.keras.Sequential([
            
            efn.EfficientNetB5(
                input_shape=(IMSIZE[IMS], IMSIZE[IMS], 3),
                weights='imagenet',
                include_top=False),
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dropout(0.4),
            tf.keras.layers.Dense(n_labels, activation='sigmoid')
        ])
        model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            #optimizer = tfa.optimizers.RectifiedAdam(learning_rate=config['start_lr']),
            #optimizer=tfa.optimizers.Lookahead(
            #    #tf.keras.optimizers.Adam(learning_rate=config['start_lr']),
            #    tfa.optimizers.RectifiedAdam(learning_rate=config['start_lr']),
            #    sync_period=6,
            #    slow_step_size=0.5
            #),
            loss='binary_crossentropy',
            metrics=[tf.keras.metrics.AUC(multi_label=True)])
            #metrics=[tf.keras.metrics.AUC()])

        model.summary()


    steps_per_epoch = train_paths.shape[0] // BATCH_SIZE
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        f'model{i}.h5', save_best_only=True, monitor='val_loss', mode='min')
    lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", patience=3, min_lr=config["min_lr"], mode='min')

    history = model.fit(
        train_dataset, 
        epochs=config["epoch"],
        verbose=1,
        callbacks=[checkpoint, lr_reducer, WandbCallback()],
        steps_per_epoch=steps_per_epoch,
        validation_data=valid_dataset)

    hist_df = pd.DataFrame(history.history)
    hist_df.to_csv(f'history{i}.csv')
    wandb.finish()